# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner and combine it with our other data skills. In this lab you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer follow-up questions. 


### Outline:

1. Determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create Python functions to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results.

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- What are the 5 businesses with the highest average ratings?
- What are the 5 businesses with the lowest average ratings?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- How many businesses have a rating above 4.5?
- How many businesses have a rating below 3?
- Return the text of the oldest review in the table.
- Return the overall rating of the business with the oldest review. 
- Find the highest rated business and return text of the newest review of the three.
- Find the lowest rated business and return text of the newest review of the three.  


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [78]:
## Connect to DB server on AWS
import mysql.connector
from mysql.connector import errorcode
import time
import pandas as pd
import config1
config1.host

'yelppp.cdhanbzj0dic.us-east-2.rds.amazonaws.com'

In [79]:
cnx = mysql.connector .connect(
    host = config1.host,
    user = config1.user,
    passwd = config1.password,
)

In [80]:
cursor = cnx.cursor()
db_name = 'yelp3'

In [81]:
def create_database(cursor, database): # cursor created above and database to be used
    # this creates the data base
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

# this will create a database if it does not work
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

Database yelp3 does not exists.
Database yelp3 created successfully.


In [82]:
## Create new DB
cnx = mysql.connector .connect(
    host = config1.host,
    user = config1.user,
    passwd = config1.password,
    database = 'yelp3'
)

print(cnx) # it will print a connection object if everything is find

In [83]:
cursor = cnx.cursor()

In [84]:
def drop_everything():
    cursor.execute("""DROP TABLE Businesses;""")
    cnx.commit()

In [85]:
# Create a table for the Businesses - We did that on mysql
cursor.execute("""CREATE TABLE IF NOT EXISTS businesses (business_id TEXT, alias TEXT, name TEXT, review_count INTEGER, rating FLOAT, price TEXT)""")

In [86]:
# Create a table for the reviews - We did that on mysql


## Part 2: Create ETL pipeline for the business data from the API

In [87]:
import json
import requests
import time
# f = open('data.jason')
# data = json.load(f)

In [88]:
term = 'italian'
location = 'New York NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }

In [89]:
api_key = 'apC6W5V_e5MB6oVHzSFz7CRmKSaVKBDWlNDWrIhZDwwYHOyUs5abXwKLmxSf6M1CExNI4fYPLBKq6xVhuTS0TGygAypfTT9feHgDbi4xnOY3VTb3Vt1LySr1qfY5XXYx'

In [90]:
# write a function to make a call to the API
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [91]:
data_italian = yelp_call(url_params, api_key)

In [92]:
len(data_italian[0])

16

In [93]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB
def all_results(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 100:
        url_params['offset'] = cur # number I am at to offset the restults
        # you can parse to avoid appending
        results.extend(yelp_call(url_params, api_key))
        time.sleep(1) #Wait a second
        cur += 50
    return results

In [94]:
data_italian_all = all_results(url_params, api_key)

8200 total matches found.


In [95]:
len(data_italian_all)

100

In [96]:
first_resto = data_italian_all[0]
first_resto

{'id': 'DoSU8IPq-Py_YV3kYmXPfQ',
 'alias': 'trattoria-trecolori-new-york',
 'name': 'Trattoria Trecolori',
 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/G8vBgHxbEfoONO0JpLX3Qw/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/trattoria-trecolori-new-york?adjust_creative=txnhmZGNUmhSEfHSekziLQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=txnhmZGNUmhSEfHSekziLQ',
 'review_count': 2162,
 'categories': [{'alias': 'italian', 'title': 'Italian'}],
 'rating': 4.0,
 'coordinates': {'latitude': 40.75998, 'longitude': -73.98671},
 'transactions': ['restaurant_reservation', 'pickup', 'delivery'],
 'price': '$$',
 'location': {'address1': '254 West 47th St',
  'address2': '',
  'address3': '',
  'city': 'New York',
  'zip_code': '10036',
  'country': 'US',
  'state': 'NY',
  'display_address': ['254 West 47th St', 'New York, NY 10036']},
 'phone': '+12129974540',
 'display_phone': '(212) 997-4540',
 'distance': 6083.735166572698}

In [64]:
first_resto['id']

'DoSU8IPq-Py_YV3kYmXPfQ'

In [98]:
test = [data_italian_all[0], data_italian_all[1]]

In [104]:
test1 = [data_italian_all[2], data_italian_all[3]]
test1

[{'id': '16ZnHpuaaBt92XWeJHCC5A',
  'alias': 'olio-e-piú-new-york-3',
  'name': 'Olio e Piú',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/l4peUZMi7CLFcG3w3jnYvQ/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/olio-e-pi%C3%BA-new-york-3?adjust_creative=txnhmZGNUmhSEfHSekziLQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=txnhmZGNUmhSEfHSekziLQ',
  'review_count': 1845,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'},
   {'alias': 'italian', 'title': 'Italian'},
   {'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}],
  'rating': 4.0,
  'coordinates': {'latitude': 40.733826, 'longitude': -73.999852},
  'transactions': ['restaurant_reservation', 'pickup', 'delivery'],
  'price': '$$',
  'location': {'address1': '3 Greenwich Ave',
   'address2': None,
   'address3': '',
   'city': 'New York',
   'zip_code': '10014',
   'country': 'US',
   'state': 'NY',
   'display_address': ['3 Greenwich Ave', 'New York, NY 10014']},
  'ph

In [107]:
if 'price' in test[0].keys():
    price1 = test[0]['price']
else:
    price1 = ''
price1

'$$'

In [99]:
def dynamic_data_entry(resturant):
    business_id = resturant['id']
    alias = resturant['alias']
    name = resturant['name']
    review_count = resturant['review_count']
    rating = resturant['rating']
    if 'price' in resturant.keys():
        price = resturant['price']
    else:
        price = ''
        
    
    cursor = cnx.cursor()
    cursor.execute("""INSERT INTO businesses (business_id, alias, name, review_count, rating, price) 
                          VALUES (%s, %s, %s, %s, %s, %s)""", (business_id, alias, name, review_count, rating, price))
    cnx.commit()
    
for i in list(range(len(test))):
    dynamic_data_entry(test[i])
    

In [100]:
cursor.execute("""SELECT * FROM businesses LIMIT 10;""")
df = pd.DataFrame(cursor.fetchall())
df

,0,1,2,3,4,5
0,DoSU8IPq-Py_YV3kYmXPfQ,trattoria-trecolori-new-york,Trattoria Trecolori,2162,4.0,$$
1,rp8S7scxOkVVeG7dNneMGQ,l-artusi-new-york,L'Artusi,1722,4.5,$$$


In [1]:
# Write a function to take your parsed data and insert it into the DB

## Part 3: Create ETL pipeline for the review data from the API

In [ ]:
# write a query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [ ]:
# write a function that take a business id 
# and makes a call to the API for reivews
# then parse out the relevant information

In [ ]:
# write a function to insert the parsed data into the reviews table

In [ ]:
## Part 4: Write SQL queries that will answer the questions posed. 

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter.***

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 100:
        url_params['offset'] = cur # number I am at to offset the restults
        # you can parse to avoid appending
        results.append(yelp_call(url_params, api_key))
        time.sleep(1) #Wait a second
        cur += 50
    return results

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id VARCHAR(30) NOT NULL,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```